---
Some useful $\LaTeX$ commands are defined in this cell:
$$
\newcommand{\abs}[1]{\left\lvert#1\right\rvert}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\paren}[1]{\left(#1\right)}
\newcommand{\brack}[1]{\left[#1\right]}
\newcommand{\ip}[2]{\left\langle#1,#2\right\rangle}
\DeclareMathOperator{\span}{span}
\DeclareMathOperator{\fl}{fl}
\abs{x}, \norm{x}, \set{x}, \paren{x}, \brack{x}, \ip{x}{y}, \span, \fl
$$

---

---
# 16.1 Initial value ordinary differential equations
---

## Ordinary differential equations and initial value problems

Many important applied problems can be posed as finding a function $y(t)$ that satisfies an **ordinary differential equation (ODE)**:

$$ \frac{dy}{dt} = f(t, y), \quad a \leq t \leq b. $$

That is, the slope of the graph of the function $y = y(t)$ at time $t$ must be given by $f(t, y)$.

An ODE with an initial condition $y(a) = c$ is called an **initial value ODE problem**:

$$ \frac{dy}{dt} = f(t, y), \quad a \leq t \leq b, \quad y(a) = c. $$

---

## Example

Consider the ODE with $f(t,y) = -5y + 5t^2 + 2t$ over the interval $[0,1]$:

$$\frac{dy}{dt} = -5y + 5t^2 + 2t, \quad 0 \leq t \leq 1.$$

Verify that for any scalar $\alpha$ the function

$$y(t) = t^2 + \alpha e^{-5t}$$

satisfies this ODE.

Given an additional condition that $y(0) = c$, we can determine that $\alpha = c$.


Check that $y(t)$ satisfies the differential equation:

$$
y'(t) = 2t - 5 \alpha e^{-5t}
$$

$$
\frac{dy}{dt} = -5 (t^2 + \alpha e^{-5t}) + 5t^2 + 2t
$$

$$
\frac{dy}{dt} = -5t^2 - 5 \alpha e^{-5t} + 5t^2 + 2t
$$

$$
\frac{dy}{dt} = -5 \alpha e^{-5t} + 2t
$$

In [ ]:
# ODE is dy/dt = f(t, y)
f(t, y) = -5y + 5t^2 + 2t

In [ ]:
using PyPlot

a, b, N = 0, 1, 20

# Create the t and y meshes of points
tt = range(a, b, length=N)
yy = range(a, b, length=N)

# dy/dt = f(t, y)
dt = [1.0 for y in yy, t in tt]
dy = [f(t, y) for y in yy, t in tt]

# Normalize arrow lengths
r = (dt.^2 + dy.^2).^0.5
u, v = dt./r, dy./r

# Create the slope field plot
quiver(tt, yy, u, v, headwidth=0)
ylabel(L"y")
xlabel(L"t");

# Plot some solution trajectories
tspan = range(a, b, length=100)
for c = 0:.2:1
    plot(tspan, tspan.^2 + c*exp.(-5tspan))
end
xlim(0, 1)
ylim(0, 1);

---

## Numerical solution of initial value problems

In this chapter we study numerical methods to approximate the function $y(t)$ that solves the initial value problem

$$ \frac{dy}{dt} = f(t, y), \quad a \leq t \leq b, \quad y(a) = c $$

at evenly spaced points $a = t_0 < t_1 < \cdots < t_N = b$, where 

$$h = \frac{b - a}{N}, \quad t_i = a + ih, \quad i = 0, 1, \ldots, N.$$

We will compute $y_i$ such that $y_i \approx y(t_i)$, for all $i$.

---

## Example

Consider the initial value problem

$$\frac{dy}{dt} = -5y + 5t^2 + 2t, \quad 0 \leq t \leq 1, \quad y(0) = 1.$$

The solution of this problem is $y(t) = t^2 + e^{-5t}$. We will approximate this solution using the Runge-Kutta (fourth order) method.

In [ ]:
using ODE

In [ ]:
# ODE is dy/dt = f(t, y)
f(t, y) = -5y + 5t^2 + 2t

a, b, N = 0, 1, 10

y0 = 1.0

tspan = range(a, b, length=N+1)

In [ ]:
y(t) = t^2 + exp(-5t)

In [ ]:
# Numerically solve the ODE using the 
# Runge-Kutta (fourth order) method
tout, yout = ode4(f, y0, tspan)

# Compute the exact solution
ytrue = y.(tout)

[tout yout ytrue]

In [ ]:
# Compute the absolute error
abserr = abs.(yout - ytrue)

```
N = 10   =>  h = 0.1   =>  abserr = 1e-4
N = 100  =>  h = 0.01  =>  abserr = 1e-8
```

Therefore, `ode4` is $\mathcal{O}(h^4)$.

In [ ]:
tt = range(a, b, length=100)

subplot(2, 1, 1)
plot(tt, y.(tt), label="true soln")
plot(tout, yout, ".", label="approx soln")
xticks([])
title("Runge-Kutta (fourth order) method")
legend()

subplot(2, 1, 2)
plot(tout, abserr)
title("Numerical solution error");

---

## ODE systems

Both the function $y(t)$ and $f(t, y)$ can be **vector-valued** functions, which gives us an ODE system, such as

$$
y' = \begin{bmatrix} y_1' \\ y_2' \end{bmatrix} = \begin{bmatrix} f_1(t, y) \\ f_2(t, y) \end{bmatrix} = f(t, y).
$$

This is especially useful for expressing higher order differential equations as a first order ODE system.

---

## Example

The friction-free motion of a swinging pendulum of length $r$ is given by the second-order initial value problem

$$
\frac{d^2 \theta}{dt^2} + \frac{g}{r} \sin \theta = 0, \quad \theta(0) = c_0, \quad \theta'(0) = c_1,
$$

where $\theta(t)$ is the vertical angle of the pendulum and $g \approx 9.8~\mathrm{m/s^2}$ is the accelation due to gravity.

If we let $y_1(t) = \theta(t)$ and $y_2(t) = \theta'(t)$, then

$$
y' = \begin{bmatrix} y_1' \\ y_2' \end{bmatrix} = \begin{bmatrix} y_2 \\ -\frac{g}{r}\sin y_1 \end{bmatrix} = f(t, y), \quad y(0) = \begin{bmatrix} c_0 \\ c_1 \end{bmatrix}.
$$

In [ ]:
g = 9.8
r = 1.0

# Without friction
f(t, y) = [y[2]; -g/r*sin(y[1])]

# With friction
f(t, y) = [y[2]; -g/r*sin(y[1]) - 2y[2]]

In [ ]:
a, b, N = 0, 10, 100

tspan = range(a, b, length=N+1)

In [ ]:
y0 = [pi/4, 2]

# Numerically solve the ODE system
tout, yout = ode4(f, y0, tspan)

yout

In [ ]:
θ = [y[1] for y in yout]

In [ ]:
plot(tout, θ)
xlabel(L"t")
ylabel(L"\theta");

---